In [1]:
from serial import Serial
from serial import SerialException
import time
import modbus_crc
ser=Serial(port='COM4', baudrate=250000)
gocode="G90"
ser.write(gocode.encode())
se=Serial(port='COM13', baudrate=9600)

In [2]:
#初始化三轴和注射泵
#gcode= "G28 X0 Y0 Z20"  # 例如，发送G28命令使打印机归位(Z的大小视情况而定)
#ser.write(gcode.encode())
#time.sleep(1)
#gcode ="G21"  # 设置单位为毫米
#ser.write(gcode.encode())
#time.sleep(1)


In [2]:
def move_to_destination(x,y,z):
    ser.flushInput()
    data_to_send=f"G00 X{x} Y{y} Z{z}\n"
    ser.write(data_to_send.encode())
def return_home():
    data_to_send="G28 X0 Y0 Z0"
    time.sleep(1)
    ser.write(data_to_send.encode())
def read_information():
    while True:
        try:
            # 读取一行数据
            line = ser.read(15).decode().strip()
            if line:  # 如果接收到了数据
                print("Received:", line)
                return 1
                break  # 读取到数据后退出循环
            else:
                time.sleep(0.1)  # 稍作延时，避免CPU占用过高
        except SerialException as e:
            print("Serial error:", e)
            return 0
            break  # 如果串口发生错误，退出循环
            
def add_crc(data):
        data_to_send=data+bytes(modbus_crc.crc16(data))
        return data_to_send
def transform_distance(distance,data_distance):
    data_distance.append(distance//256)
    data_distance.append(distance%256)
    data_distance_to_send=add_crc(bytes(data_distance))
    return data_distance_to_send
def chouqu(se):
    se.flushInput()
    distance_=eval(input("请输入抽取体积(250代表2ml)："))
    data_distance=[0x03,0x06,0x00,0x07]
    data_distance_to_send=transform_distance(distance_,data_distance)
    se.write(data_distance_to_send)
    time.sleep(1)
    data_speed=[0x03,0x06,0x00,0x05,0x00,0xFF]
    data_speed_to_send=add_crc(bytes(data_speed))
    se.write(data_speed_to_send)
    time.sleep(1)
    data_zhengzhuan=[0x03,0x06,0x00,0x00,0x00,0x01]
    data_zhengzhuan_to_send=add_crc(bytes(data_zhengzhuan))
    se.write(bytes(data_zhengzhuan_to_send))
def zhushe(se):
    distance_=eval(input("请输入注射体积(250代表2ml)："))
    data_distance=[0x03,0x06,0x00,0x07]
    data_distance_to_send=transform_distance(distance_,data_distance)
    se.write(data_distance_to_send)
    time.sleep(1)
    data_speed=[0x03,0x06,0x00,0x05,0x00,0xFF]
    data_speed_to_send=add_crc(bytes(data_speed))
    se.write(data_speed_to_send)
    time.sleep(1)
    se.flushInput()
    data_fanzhuan=[0x03,0x06,0x00,0x01,0x00,0x01]
    data_fanzhuan_to_send=add_crc(bytes(data_fanzhuan))
    se.write(bytes(data_fanzhuan_to_send))
def main():
    x=input("请输入x方向移动距离(mm):")
    y=input("请输入y方向移动距离(mm):")
    z=input("请输入z方向移动距离(mm):")
    move_to_destination(x,y,z)


In [4]:
main()#移动到指定移液位置的上方

In [5]:
chouqu(se)

In [6]:
zhushe(se)

In [7]:
main()#将枪头伸入离心管中
if read_information():#回读信息
    chouqu(se)
main()#回到指定移液位置的上方
return_home()